In [1]:
import run_api

Obtained my token successfully!
{'data': 1, 'error': 'success', 'error_Description': 'Task is started successfully!'}
{'data': 1, 'error': 'success', 'error_Description': 'Task is started successfully!'}
{'data': 1, 'error': 'success', 'error_Description': 'Task is started successfully!'}
{'data': 1, 'error': 'success', 'error_Description': 'Task is started successfully!'}


In [2]:
import utils
from ner import NER

csv = utils.csv
pd = utils.pd
np = utils.np
nltk = utils.nltk
itertools = utils.itertools
re = utils.re

[nltk_data] Downloading package stopwords to /home/kasem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kasem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
xls = pd.ExcelFile('strings stems.xlsx')

first = pd.read_excel(xls, 'first').dropna(axis=1, how='all').dropna(axis=0, how='all')
parent_second = pd.read_excel(xls, 'parent second').dropna(axis=1, how='all').dropna(axis=0, how='all')
bad_keywords= pd.read_excel(xls, 'bad keywords').dropna(axis=1, how='all').dropna(axis=0, how='all')

tutoring = pd.read_excel(xls, 'Tutoring').dropna(axis=1, how='all').dropna(axis=0, how='all')
payments = pd.read_excel(xls, 'payments').dropna(axis=1, how='all').dropna(axis=0, how='all')
contract = pd.read_excel(xls, 'contract').dropna(axis=1, how='all').dropna(axis=0, how='all')

firstextract_exported = pd.read_csv("firstextract.csv")
secondextract_exported = pd.read_csv("secondextract.csv")

utils.pre_process_df_keywords(first)
utils.pre_process_df_keywords(parent_second)
utils.pre_process_df_keywords(bad_keywords)


In [4]:
extractor = NER()

In [8]:
Nfiles_df_list, first_extract, second_extract, third_extract = run_api.Nfiles_df_list, run_api.first_extract_df, run_api.second_extract_df, run_api.third_extract_df
first_extract.rename(inplace=True, columns={"field1_Text_Text": "title", "field1_Link_Link": "link"})
first_extract = run_api.cleanup_df(first_extract,"title")
second_extract = run_api.cleanup_df(second_extract,"field1")
third_extract = run_api.cleanup_df(third_extract,"email")

extracted_pay = utils.extract_pay_info(second_extract,extractor)

In [18]:
second_extract_pay_appended = second_extract.reset_index().merge(extracted_pay.reset_index(), on="index",how="left").drop(["index","field","Res"],axis=1)
all_info = second_extract_pay_appended.merge(third_extract, on= "field1", how="left").dropna().drop_duplicates(subset = "email")
all_info.rename(columns={"field1": "title", "field2":"body", "field3": "compensation","email": "email", "Decesion":"pay_amount"}, inplace=True)

all_info.to_csv("sent.csv", index= False)
all_info

,title,body,compensation,pay_amount,amount,email
0,Seeking MILF Pick-up Artists,QR Code Link to This Post\n \n ...,no pay,-1,NA,8b931e0f988a32cdad01923aa103cd78@gigs.craigsli...
1,Sr. Business Analyst,QR Code Link to This Post\n \n ...,compensation: Competative Hourly Rate,-1,hourly,948f2dc226e53ae7963cb1681fc5fd15@gigs.craigsli...
2,Clothing fitting model and feedback needed,QR Code Link to This Post\n \n ...,compensation: TBD,-1,NA,ce31860047f33c4183513bab9d08da22@gigs.craigsli...
3,CAD & Graphic Rendering and layout,QR Code Link to This Post\n \n ...,compensation: Starting at $20.00 per hour base...,20,hour,37a5a1e9d67f3bb78038e98b0b6dbc09@gigs.craigsli...
4,Public Speaking / Retail Presenter W2 $700-120...,QR Code Link to This Post\n \n ...,compensation: W2 Guaranteed weekly draw/commis...,-1,commissions,7d176b37ffa4305984556698216dd6e5@gigs.craigsli...
...,...,...,...,...,...,...
183,Models needed for Product Photoshoot,QR Code Link to This Post\n \n ...,compensation: $50 for 2 hours,50,hour,c2514bfb979e33acb87857ce9e7aaacf@gigs.craigsli...
184,Haircut Model Casting,QR Code Link to This Post\n \n ...,compensation: $100,100,NA,2d6acdfe1aa0314bbbd4e23765a4b382@gigs.craigsli...
185,sound mixer/boom operator with equipment,QR Code Link to This Post\n \n ...,compensation: $100/day,100,day,4208321e431b3a5f87df33ed02b65f86@gigs.craigsli...
186,Looking For Festival Models,QR Code Link to This Post\n \n ...,compensation: $15-$50+/hr (contract),15,hr,1ed3f5fd1d86383a91036a30d3ac002e@gigs.craigsli...


In [16]:
second_extract[second_extract["field1"] == "Female models start today!!"]

,field1,field2,field3
187,Female models start today!!,QR Code Link to This Post\n \n ...,compensation: $1300 per week or more


'hi, hsadda'